First we'll import the necessary functions 

In [7]:
#imports and functions
import pke
import string
import pandas as pd
import random
import matplotlib.pyplot as plt
from clean_text import clean 
import re
from nltk import word_tokenize



#Function searches keyword list and highlights them in the text
from IPython.display import HTML
import re

def highlight_keywords(text, keywords):
    toreplace = "<b><u>\g<0></u></b>"
    for word in keywords:
        pattern = re.compile(re.escape(word), re.I)
        highlighted_txt = re.sub(pattern,toreplace,text)
        text = highlighted_txt
    return HTML(highlighted_txt)

#Function returns keyphrases from tuple output
def get_keyphrases(keyphrases_tuples):
    output_list = []
    for entry in range(len(keyphrases_tuples)):
        output_list.append(keyphrases_tuples[entry][0])
    return output_list

#Function generates highlighted text for all graph methods
def generate_all_outputs_tr(text_df, random_indexes):
    for number in random_indexes:
        print("---------------Entry: -----------------")
        text = clean(text_df['threads'][number])
        extractor_TR = pke.unsupervised.TextRank()
        extractor_TR.load_document(input=text, language='en')
        extractor_TR.candidate_selection()
        extractor_TR.candidate_weighting()
        
        if len(text) >= 100:
            optimal_keyphrases = int(round(len(text) / 100, 0))
        else:
            optimal_keyphrases = 1
        keyphrases_TR = extractor_TR.get_n_best(n=optimal_keyphrases)

        output = get_keyphrases(keyphrases_TR)
        display(highlight_keywords(text, output))
        print("\n")


def generate_all_outputs_pr(text_df, random_indexes):
    for number in random_indexes:
        print("---------------Entry: -----------------")
        text = clean(text_df['threads'][number])
        extractor_TR = pke.unsupervised.PositionRank()
        extractor_TR.load_document(input=text, language='en')
        extractor_TR.candidate_selection()
        extractor_TR.candidate_weighting()
        if len(text) >= 100:
            optimal_keyphrases = int(round(len(text) / 100, 0))
        else:
            optimal_keyphrases = 1
        keyphrases_TR = extractor_TR.get_n_best(n=optimal_keyphrases)
        output = get_keyphrases(keyphrases_TR)
        display(highlight_keywords(text, output))
        print("\n")

def generate_all_outputs_topicRank(text_df, random_indexes):
    for number in random_indexes:
        print("---------------Entry: -----------------")
        text = clean(text_df['threads'][number])
        extractor_TR = pke.unsupervised.TopicRank()
        extractor_TR.load_document(input=text, language='en')
        extractor_TR.candidate_selection()
        extractor_TR.candidate_weighting()
        if len(text) >= 100:
            optimal_keyphrases = int(round(len(text) / 100, 0))
        else:
            optimal_keyphrases = 1
        keyphrases_TR = extractor_TR.get_n_best(n=optimal_keyphrases)
        output = get_keyphrases(keyphrases_TR)
        display(highlight_keywords(text, output))
        print("\n")


def generate_all_outputs_mr(text_df, random_indexes):
    for number in random_indexes:
        print("---------------Entry: -----------------")
        text = clean(text_df['threads'][number])
        extractor_TR = pke.unsupervised.MultipartiteRank()
        extractor_TR.load_document(input=text, language='en')
        extractor_TR.candidate_selection()
        extractor_TR.candidate_weighting()
        if len(text) >= 100:
            optimal_keyphrases = int(round(len(text) / 100, 0))
        else:
            optimal_keyphrases = 1
        keyphrases_TR = extractor_TR.get_n_best(n=optimal_keyphrases)
        output = get_keyphrases(keyphrases_TR)
        display(highlight_keywords(text, output))
        print("\n")







        
#Fixes issues with formatting
def Convert(string): 
    try:
        li = list(string.split(", "))
    except:
        li = ""
    return li 
        

#Evaluation
def exact_get_scores(keyphrases, ngsl_output, tsl_output):
    # tokenize keyphrases to split up multi-words?
    keyphrases = word_tokenize(" ".join(keyphrases))
   # print("HERE", keyphrases)

  
    ngsl_score = len(set(keyphrases).intersection(
        ngsl_output)) / len(ngsl_output)
    if len(tsl_output) > 0:
        tsl_score = len(set(keyphrases).intersection(
            tsl_output)) / len(tsl_output)
    else:
        tsl_score = 1
    print('ngsl: {} out of {}'.format(len(set(keyphrases).intersection(
        ngsl_output)), len(ngsl_output)))
    print('tsl: {} out of {} \n'.format(len(set(keyphrases).intersection(
            tsl_output)), len(tsl_output)))
    print('ngsl accuracy: {}% \ntsl accuracy: {}%'.format(round(ngsl_score*100,2), round(tsl_score*100, 2)))
    

def get_recall(keyphrases, ngsl_output, tsl_output):
    keyphrases = word_tokenize(" ".join(keyphrases))

    ngsl_tp = len(set(keyphrases).intersection(
        ngsl_output))

    ngsl_fn = len([word for word in ngsl_output if word not in keyphrases])
    try:
        ngsl_recall = round((ngsl_tp / (ngsl_fn + ngsl_tp))*100, 2)
    except:
        ngsl_recall = math.nan

    tsl_tp = len(set(keyphrases).intersection(
        tsl_output))

    #Get keyphrases from graph output that were not in the tsl_outputs
    #That means, since the tsl_output is based on text, that it should have been selected as a keyphrase
    tsl_fn = len([word for word in tsl_output if word not in keyphrases])

    try:
        tsl_recall = round((tsl_tp / (tsl_fn + tsl_tp))*100, 2)
    except:
        tsl_recall = math.nan

    print('ngsl recall: {}% \ntsl recall: {}%\n'.format(
        ngsl_recall, tsl_recall))

    return ngsl_recall, tsl_recall

Then we'll load up our dataset. For simplicity, I randomly select 5 threads and change the index variable to cycle through them. Note that as of now, I have not concentated the post to the threads but I think moving forward it will be something I'll do

In [8]:
#Read dataset and generate 5 random samples
text_df = pd.read_csv("../Data/sample_data.csv", encoding="utf-8")
random.seed(56)
random_indexes = random.sample(range(0, len(text_df)), 5)

In [9]:
index = 4

Minor cleaning of text such as removing urls, emoji's and white spaces. There are a couple of issues that need to fix. 

<ol> 
    <li> The regular expression for the urls does not always work </li>
    <li> I currently don't do any 'correcting' of misspellings. This includes both grammatical and structural ones such as <b>two words stuck together </b> </li>
    <li> A major one: I don't clean the text before labeling the data which skews the results </li>
    <li> I have not done any filtering of the dataset I'm using. This means that there are spanish posts and very sparse entries present (example: 1-4 words only) </li>
</ol>

Also note, the removal of stopwords is done during the candidate selection phase of the graph algorithm


In [17]:

def clean(text):
    
    # remove see more
    text = re.sub(r' ?\w*?[…]see more.', '', text, flags=re.I)
    
    # remove urls
    text = re.sub(r'^http?://.*[rn]*|www.\S+', '', text, flags=re.MULTILINE)

    # remove emojis
    text = remove_emoji(text)

    # remove special characters
    text = remove_special_characters(text
                                     )


    # all lower case
    text = "".join([w.lower() for w in text])
    

    # remove extra newlines
    text = re.sub(r'[\r|\n|\r\n]+', ' ', text)

    # remove extra whitespace
    text = re.sub(' +', ' ', text)

    return text

# From https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


# from https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text



In [18]:
text = clean(text_df['threads'][random_indexes[index]])

In [19]:
text

'i remember my first student evaluations 1 more short skirts 2 your hair looks so much better down 3 the psych building is too far away from my dorm room i decided at this point that student evaluations are not the definitive critique of my teaching kristy m keefe my first semester teaching i got two comments saying to wear less clothes as a new instructor this was not what i was expecting and not helpful in how i could improve as an educator sadly this may have been the first time i received comments about my appearance but it was not the last ugh thats sexual harassment i got one saying someone should make a calendar with pictures of my sexy butt literally kristy m keefe i had the opposite for the hair my hair is long and i always wear it down student said that i either needed to cut my hair or wear it in a bun because long hair is unprofessional students will reach for anything when they want to be negative one time a student asked me a question about the midterm and i said i hadnt 

For this demonstration I'll show textRank and TopicRank. Note for choosing the number of keyphrases to generate, I went with returning 2.5% of the total characters (or just 1 if not large enough). Upon closer examination, I think that this should definitely <b>be increased </b>. There is existing literature that comments on this which I will consult as an appriopate start, but it is something to play around with

In [8]:
extractor = pke.unsupervised.TextRank()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()

if len(text) >= 100:
    optimal_keyphrases = int(round(len(text) / 100, 0))
else:
    optimal_keyphrases = 1

keyphrases = extractor.get_n_best(n=optimal_keyphrases) 

In [9]:
keyphrases

[('divisive ideology juan trucupey cool story bro catherine glass',
  0.03766659047826),
 ('usee more harry green brilliant bob borrelli', 0.035433624229812784),
 ('lines graphic sexual fantasies sexual assault', 0.030506592066616864),
 ('lol lisa parker lebovitz daniel malcom thanks', 0.027250913004578662),
 ('better qualified white males', 0.02700476851371925),
 ('obvious ones student interest', 0.025980624888498692),
 ('good angry people', 0.025759594202689286),
 ('stupid question people amanda evert', 0.02552002577453611),
 ('first student evaluations', 0.025221561259155284),
 ('good reviews', 0.02478698536567541),
 ('wrong isee more propaganda', 0.024130483469775567),
 ('bias eg white men', 0.02349773129165103),
 ('few good ones', 0.022953466317944166),
 ('good professor', 0.02285878365793789),
 ('good teaching _', 0.022848112780077378),
 ('people good', 0.022560420304301276),
 ('shit ami sedani new study', 0.022366715454889247),
 ('young white male faculty', 0.02155580536005585),

Here is the output with the extracted keyphrases underlined. This method is not perfect as it will highlight text that does not 100% match the keyphrase, but it is sufficent for now. As demonstrated by the output, extracting 1% of keyphrases relative to the size of the text is may be too much, but this is not conclusive

One main issue I have with this implementation of TextRank is that the number of words for a keyphrase tends to be on the longer side. I'm not sure if this can be adjusted but for our implementation I think we'd liked to stick mostly around 1-2 words with occasional 3 word keyphrases and a rare 4 word keyphrases.


In [10]:
output = get_keyphrases(keyphrases)
highlight_keywords(text, output)

ngsl = new general service list. The purpose of this word list was to gather 2,800 most important words in english which is determined by their frequency in text. This particular word list is an update to the original gsl published in 1953. The overlap between the two is large, and this new list is more culturally relative to now.

The audience for this word list is English Language Learners

I think these labels are perfect to model 'tier 1 words' which are words that are very frequent and basic. 
<b> Please note: </b> As a pre-process step, I remove all stopwords and the word "I'm". This is about 100-200 words removed.

Link to wordlist and explanation - http://www.newgeneralservicelist.org

In [11]:
output_ngsl = Convert(text_df['ngsl_labels'][random_indexes[index]])
output_ngsl_list = []
for word in output_ngsl:
    output_ngsl_list.append(word)

print(output_ngsl_list)

highlight_keywords(text, output_ngsl_list[0:7]) #given the number of ngsl words, I limited to the top 8 


['improve', 'student', 'well', 'whole', 'college', 'several', 'stupid', 'avoid', 'happen', 'though', 'teach', 'could', 'someone', 'come', 'typical', 'stuff', 'class', 'gender', 'quality', 'make', 'doubt', 'also', 'bias', 'less', 'comment', 'expect', 'time', 'heavy', 'male', 'without', 'always', 'opposite', 'learn', 'strong', 'positive', 'ideal', 'female', 'bad', 'either', 'text', 'put', 'may', 'attention', 'hate', 'get', 'part', 'mean', 'complete', 'destroy', 'sexual', 'away', 'professor', 'principle', 'must', 'yeah', 'largely', 'new', 'white', 'assumption', 'study', 'story', 'care', 'found', 'young', 'every', 'great', 'decide', 'improvement', 'anything', 'direct', 'mental', 'say', 'disappoint', 'include', 'actually', 'attractive', 'laugh', 'cause', 'perhaps', 'totally', 'interest', 'one', 'standard', 'fear', 'helpful', 'hair', 'believe', 'lot', 'submit', 'reach', 'nice', 'amount', 'link', 'paper', 'different', 'specific', 'cut', 'good', 'still', 'spend', 'field', 'ever', 'remind', 'li

TSL word list = TOIEC Service List

These model a mixture of lower level tier 2 words. The size of this word list is not very large. From the website linked above, it is stated that a combination of ngsl and TSL accounts for 99% of the tested material on the TOIEC.

There are other word lists that I'm working on unpacking. Most notably is the academic word list which also models tier 2 words, but to a larger degree. The difference between TOEIC and AWL is that TOIEC is marketed towards english language learners specifically and AWL is more general. 

The focus on the system should be to have a good performance on tier 2 words. There are word lists for tier 3 words, but because they are seperated by a bunch of different subjects (biology, chemistry, law) and their low frequency, the time tradeoff isn't good for right now to invest time on that yet

I think this will also be interesting to see how the different parameters and filtering impact the types of words the system extracts based on word difficulty. To integrate word difficulty later, all we would have to do is use the frequency of each word which is already a part of the data

In [12]:
output_tsl = Convert(text_df['tsl_labels'][random_indexes[index]])
output_tsl_list = []
for word in output_tsl:
    output_tsl_list.append(word)

if len(output_tsl_list) > 0:
    print(output_tsl_list)
    display(highlight_keywords(text, output_tsl_list))
else:
    print("No TSL words")

['criteria', 'popularity', 'profitable', 'contrary', 'pharmacy', 'inappropriate', 'instructor', 'poorly', 'calendar', 'clap', 'candidate', 'semester', 'worthwhile', 'graphic', 'feedback', 'merit', 'faculty']


<b> Issue with this evaluation </b>

<ol> 
    <li> I have not yet worked out how to score partial matches. For now, before scoring each keyphrase will be seperated into individual words before scoring. This is done without any penalty  As a result, I've noticed that this implementation of textRank gets a boost because it produces longer keyphrases and thus has more chances to 'match' a desired word. </li>
    <li> I'm not sure if recall in this specific instance makes sense. Because we don't explicity label each candidate as a keyphrase or not. </li>
    <li> Once I've made the actual implementation of the graph algorithms/others, the scores will be based on the candidates themselves.  </li>
</ol>

In [13]:
get_recall(output, output_ngsl_list, output_tsl_list)
exact_get_scores(output, output_ngsl_list, output_tsl_list)

ngsl recall: 21.93% 
tsl recall: 52.94%

ngsl: 50 out of 228
tsl: 9 out of 17 

ngsl accuracy: 21.93% 
tsl accuracy: 52.94%


## Comparison to TopicRank:

In [14]:
extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()

if len(text) >= 100:
    optimal_keyphrases = int(round(len(text) / 100, 0))
else:
    optimal_keyphrases = 1

keyphrases = extractor.get_n_best(n=optimal_keyphrases) 
keyphrases

[('first student evaluations', 0.03386647089018708),
 ('comments', 0.018713831590459685),
 ('white', 0.017286181640018532),
 ('good angry people', 0.016004251548966297),
 ('male teachers', 0.015885885299095206),
 ('kantor', 0.014934364394873152),
 ('tenured hahaha attractive', 0.013177859822320776),
 ('good reviews', 0.012920804663270816),
 ('rate', 0.012906008168999863),
 ('good evaluations', 0.01216564085889058),
 ('rmp rating lol', 0.0113863012211115),
 ('bias', 0.011311575500598072),
 ('several studies', 0.011190605190669979),
 ('professor', 0.011113435937229795),
 ('courses', 0.010700045758945047),
 ('universities', 0.01065830620746337),
 ('hair', 0.010312545114013926),
 ('ones', 0.010308658184672814),
 ('undergrad subject', 0.009844567237850998),
 ('thirds', 0.009315981390089895),
 ('point', 0.008907422191618367),
 ('gender', 0.008527537689752304),
 ('direct cause', 0.00821856632089495),
 ('word', 0.007959105129148101),
 ('inferior candidates', 0.007836118200235278),
 ('first sem

In [15]:
output = get_keyphrases(keyphrases)
highlight_keywords(text, output)

In [16]:
output_ngsl = Convert(text_df['ngsl_labels'][random_indexes[index]])
output_ngsl_list = []
for word in output_ngsl:
    output_ngsl_list.append(word)

print(output_ngsl_list)

highlight_keywords(text, output_ngsl_list[0:7]) #given the number of ngsl words, I limited to the top 8 



['improve', 'student', 'well', 'whole', 'college', 'several', 'stupid', 'avoid', 'happen', 'though', 'teach', 'could', 'someone', 'come', 'typical', 'stuff', 'class', 'gender', 'quality', 'make', 'doubt', 'also', 'bias', 'less', 'comment', 'expect', 'time', 'heavy', 'male', 'without', 'always', 'opposite', 'learn', 'strong', 'positive', 'ideal', 'female', 'bad', 'either', 'text', 'put', 'may', 'attention', 'hate', 'get', 'part', 'mean', 'complete', 'destroy', 'sexual', 'away', 'professor', 'principle', 'must', 'yeah', 'largely', 'new', 'white', 'assumption', 'study', 'story', 'care', 'found', 'young', 'every', 'great', 'decide', 'improvement', 'anything', 'direct', 'mental', 'say', 'disappoint', 'include', 'actually', 'attractive', 'laugh', 'cause', 'perhaps', 'totally', 'interest', 'one', 'standard', 'fear', 'helpful', 'hair', 'believe', 'lot', 'submit', 'reach', 'nice', 'amount', 'link', 'paper', 'different', 'specific', 'cut', 'good', 'still', 'spend', 'field', 'ever', 'remind', 'li

In [17]:
output_tsl = Convert(text_df['tsl_labels'][random_indexes[index]])
output_tsl_list = []
for word in output_tsl:
    output_tsl_list.append(word)

if len(output_tsl_list) > 0:
    print(output_tsl_list)
    display(highlight_keywords(text, output_tsl_list))
else:
    print("No TSL words")

['criteria', 'popularity', 'profitable', 'contrary', 'pharmacy', 'inappropriate', 'instructor', 'poorly', 'calendar', 'clap', 'candidate', 'semester', 'worthwhile', 'graphic', 'feedback', 'merit', 'faculty']


In [18]:
get_recall(output, output_ngsl_list, output_tsl_list)
exact_get_scores(output, output_ngsl_list, output_tsl_list)

ngsl recall: 25.0% 
tsl recall: 29.41%

ngsl: 57 out of 228
tsl: 5 out of 17 

ngsl accuracy: 25.0% 
tsl accuracy: 29.41%


<b> Looking at TextRank again </b>

ngsl recall: 27.19% 
tsl recall: 52.94%

ngsl: 62 out of 228
tsl: 9 out of 17 

ngsl accuracy: 27.19% 
tsl accuracy: 52.94%

On the surface, it seems as textRank may perform better, but I don't think this is true. This is largely due to the systems evaluation error as of now. So because of this, its inconlusive to say which is truley better. At best, what we can look at right now is the type of words that are being outputted. 

In [19]:
#textRank
generate_all_outputs_tr(text_df, random_indexes)

---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------


In [20]:
#TopicRank
generate_all_outputs_topicRank(text_df, random_indexes)

---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------


In [6]:
#PositionRank
generate_all_outputs_pr(text_df, random_indexes)

---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------


In [7]:
#MultipartiteRank
generate_all_outputs_mr(text_df, random_indexes)

---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------




---------------Entry: -----------------
